# IndicF5 Gradio Demo

This notebook requires a **GPU runtime**. Go to **Runtime → Change runtime type → T4 GPU** before running.

In [ ]:
# Check GPU availability - THIS MODEL REQUIRES GPU
import torch
if not torch.cuda.is_available():
    raise RuntimeError('❌ GPU not available! Go to Runtime → Change runtime type → T4 GPU')
print(f'✅ GPU available: {torch.cuda.get_device_name(0)}')

In [ ]:
# Clear HuggingFace cache to get fresh model code
!rm -rf ~/.cache/huggingface/modules/
print('✅ HuggingFace modules cache cleared')

In [ ]:
# Clone the repository (fresh clone)
import os
!rm -rf f15  # Remove old clone
!git clone https://github.com/beginner4a3/f15
os.chdir('f15')

In [ ]:
# Install compatible library versions
%pip install -q 'transformers<4.50' accelerate
%pip install -q git+https://github.com/ai4bharat/IndicF5.git
%pip install -q gradio spaces huggingface_hub torchcodec

In [ ]:
# CRITICAL: Patch HuggingFace cached model.py to disable torch.compile
# This fixes the silent audio bug caused by dynamo compilation
import os
import glob

# Find the cached model.py
model_files = glob.glob(os.path.expanduser('~/.cache/huggingface/hub/models--ai4bharat--IndicF5/snapshots/*/model.py'))
for model_file in model_files:
    print(f'Patching: {model_file}')
    with open(model_file, 'r') as f:
        content = f.read()
    # Disable torch.compile calls
    content = content.replace('torch.compile(', '# torch.compile(')
    with open(model_file, 'w') as f:
        f.write(content)

print('✅ Model torch.compile disabled')

In [ ]:
# PATCH: Fix app.py for proper operation
with open('app.py', 'r') as f:
    content = f.read()

# 1. Disable torch.compile (causes dynamo errors)
content = content.replace(
    'if hasattr(torch, "compile"):',
    'if False:  # DISABLED'
)

# 2. Enable share=True for public link
content = content.replace(
    'iface.launch()',
    'iface.launch(share=True, debug=True)'
)

# 3. Fix deprecated autocast
content = content.replace(
    'torch.cuda.amp.autocast(enabled=True)',
    "torch.amp.autocast('cuda', enabled=True)"
)

with open('app.py', 'w') as f:
    f.write(content)

print('✅ app.py patched')

In [ ]:
# Login to Hugging Face (optional for public models)
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Run the app
!python app.py